In [52]:
#Importing the necessary libraries

from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException

In [53]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager

driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()))

TypeError: WebDriver.__init__() got an unexpected keyword argument 'service'

In [ ]:
url = 'https://www.daraz.pk/'

In [ ]:
driver.get(url)

In [47]:

search_bar = driver.find_element(By.XPATH, '/html/body/div[1]/div/div/div[1]/div/div/div[2]/div/div[2]/form/div/div[1]/input[1]')
search_bar.send_keys('Samsung' + Keys.RETURN)

WebDriverException: Message: unknown error: result.webdriverValue.value list is missing or empty in Runtime.callFunctionOn response
  (Session info: chrome=119.0.6045.160)


In [46]:
import time
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
#driver.back()
ad_grid = driver.find_element(By.XPATH, '//*[@id="root"]/div/div[3]/div/div/div[1]/div[2]')
ad_cards = ad_grid.find_elements(By.CLASS_NAME, 'gridItem--Yd0sa')
# Loop through each ad card, click on it, and then return to the previous page
price_mobile = []
dilivery_status = []
ratings_list = []
seller_ratings_list = []
Shipping_performance = []
flagship_mall = []
scores = []

for index in range(0,4):
    # Find the ad cards again to avoid StaleElementReferenceException
    ad_grid = driver.find_element(By.XPATH, '//*[@id="root"]/div/div[3]/div/div/div[1]/div[2]')
    ad_cards = ad_grid.find_elements(By.CLASS_NAME, 'gridItem--Yd0sa')

    # Click on the current ad card
    current_card = ad_cards[index]
    current_card.click()
    Information_grid = driver.find_element(By.XPATH, '/html/body/div[4]/div/div[3]')
    prices = Information_grid.find_elements(By.CLASS_NAME, 'pdp-product-price')
    price_ = Information_grid.find_element(By.XPATH, '//*[@id="module_product_price_1"]/div/div/span')
    price_mobile.append(price_.text)

    dilivery_bar = driver.find_element(By.XPATH, '//*[@id="block-Cve5yP08rW9"]')
    dilivery_opt = driver.find_elements(By.CLASS_NAME, 'delivery-option-item__title')
    Seller_rating = dilivery_bar.find_element(By.XPATH, '/html/body/div[4]/div/div[3]/div[2]/div/div[2]/div[5]/div/div[2]/div[1]/div[2]')
    Shipping_perf = dilivery_bar.find_element(By.XPATH, '//*[@id="module_seller_info"]/div/div[2]/div[2]/div[2]')
    flagship_mall_xpath = '//*[@id="module_seller_info"]/div/div[1]/div[1]/div[2]/a[2]/img'
    flagship_mall_element = dilivery_bar.find_elements(By.XPATH, flagship_mall_xpath)
    if flagship_mall_element:
        flagship_mall.append(1)
    else:
        flagship_mall.append(0)
    seller_ratings_list.append(Seller_rating.text)
    Shipping_performance.append(Shipping_perf.text)
    driver.execute_script("window.scrollTo(0, 500);")
    time.sleep(1)
    driver.execute_script("window.scrollTo(500, 800);")
    time.sleep(1)
    driver.execute_script("window.scrollTo(800, 1000);")
    time.sleep(2)
    rating_bar = driver.find_element(By.XPATH, '//*[@id="module_product_review"]')
    rating = rating_bar.find_element(By.XPATH, '//*[@id="module_product_review"]/div/div/div[1]/div[2]/div/div/div[1]/div[1]/span[1]')
    ratings_list.append(rating.text)
    counter = 0
    for i in dilivery_opt:
        if 'Free Delivery' in (i.text):
            counter+=1
            dilivery_status.append(1)
    if(counter==0):
        dilivery_status.append(0)
    
    # Calculate the base score (price)
    base_score = int(price_mobile[index].replace('Rs. ', '').replace(',', ''))

    # Check if the product has "Daraz Mall" status
    daraz_mall_score = 5000 if flagship_mall[index] == 1 else 0

    # Check if the product has free shipping
    free_shipping_score = 1000 if dilivery_status[index] == 1 else 0
    product_rating = float(ratings_list[index])
    seller_rating = float(seller_ratings_list[index].replace('%', ''))
    shipping_performance = float(Shipping_performance[index].replace('%', ''))
    score = base_score + daraz_mall_score + free_shipping_score + 10000 * (
        (0.4 * (product_rating / 5)) + (0.4 * (seller_rating / 100)) + (0.2 * (shipping_performance / 100))
    )
    
    scores.append(score)

    time.sleep(2)  # Add a short delay to allow the page to load


    # Go back to the previous page
    driver.back()
    # Wait for the ad grid to be present again to avoid StaleElementReferenceException
    WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, '//*[@id="root"]/div/div[3]/div/div/div[1]/div[2]')))
print(price_mobile)
print(dilivery_status)
print(ratings_list)
print(seller_ratings_list)
print(Shipping_performance)
print(flagship_mall)
best_product_index = scores.index(max(scores))

# Print the details of the best product
print(f"Best Product Details:")
print(f"Price: {price_mobile[best_product_index]}")
print(f"Daraz Mall Status: {'Yes' if flagship_mall[best_product_index] == 1 else 'No'}")
print(f"Free Shipping: {'Yes' if dilivery_status[best_product_index] == 1 else 'No'}")
print(f"Product Rating: {ratings_list[best_product_index]}")
print(f"Seller Rating: {seller_ratings_list[best_product_index]}")
print(f"Shipping Performance: {Shipping_performance[best_product_index]}")
print(f"Total Score: {scores[best_product_index]}")

['Rs. 51,499', 'Rs. 51,299', 'Rs. 87,999', 'Rs. 88,840']
[1, 1, 1, 1]
['4.7', '4.1', '4.0', '5.0']
['91%', '85%', '91%', '93%']
['100%', '100%', '100%', '100%']
[0, 0, 0, 0]
Best Product Details:
Price: Rs. 88,840
Daraz Mall Status: No
Free Shipping: Yes
Product Rating: 5.0
Seller Rating: 93%
Shipping Performance: 100%
Total Score: 99560.0


In [42]:
import time
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
driver.back()
ad_grid = driver.find_element(By.XPATH, '//*[@id="root"]/div/div[3]/div/div/div[1]/div[2]')
ad_cards = ad_grid.find_elements(By.CLASS_NAME, 'gridItem--Yd0sa')
# Loop through each ad card, click on it, and then return to the previous page
price_mobile = []
dilivery_status = []
ratings_list = []
seller_ratings_list = []
Shipping_performance = []
flagship_mall = []
scores = []
product_name = []
for index in range(0,2):
    # Find the ad cards again to avoid StaleElementReferenceException
    ad_grid = driver.find_element(By.XPATH, '//*[@id="root"]/div/div[3]/div/div/div[1]/div[2]')
    ad_cards = ad_grid.find_elements(By.CLASS_NAME, 'gridItem--Yd0sa')
    
    # Click on the current ad card
    current_card = ad_cards[index]
    current_card.click()
    Information_grid = driver.find_element(By.XPATH, '/html/body/div[4]/div/div[3]')
    prices = Information_grid.find_elements(By.CLASS_NAME, 'pdp-product-price')
    price_ = Information_grid.find_element(By.XPATH, '//*[@id="module_product_price_1"]/div/div/span')
    price_mobile.append(price_.text)
    

    dilivery_bar = driver.find_element(By.XPATH, '//*[@id="block-Cve5yP08rW9"]')
    dilivery_opt = driver.find_elements(By.CLASS_NAME, 'delivery-option-item__title')
    Seller_rating = dilivery_bar.find_element(By.XPATH, '/html/body/div[4]/div/div[3]/div[2]/div/div[2]/div[5]/div/div[2]/div[1]/div[2]')
    Shipping_perf = dilivery_bar.find_element(By.XPATH, '//*[@id="module_seller_info"]/div/div[2]/div[2]/div[2]')
    flagship_mall_xpath = '//*[@id="module_seller_info"]/div/div[1]/div[1]/div[2]/a[2]/img'
    name_of_product = Information_grid.find_element(By.XPATH, '//*[@id="module_product_title_1"]/div/div')
    product_name.append(name_of_product.text)
    flagship_mall_element = dilivery_bar.find_elements(By.XPATH, flagship_mall_xpath)
    if flagship_mall_element:
        flagship_mall.append(1)
    else:
        flagship_mall.append(0)
    seller_ratings_list.append(Seller_rating.text)
    Shipping_performance.append(Shipping_perf.text)
    driver.execute_script("window.scrollTo(0, 500);")
    time.sleep(1)
    driver.execute_script("window.scrollTo(500, 800);")
    time.sleep(1)
    driver.execute_script("window.scrollTo(800, 1000);")
    time.sleep(2)
    rating_bar = driver.find_element(By.XPATH, '//*[@id="module_product_review"]')
    rating = rating_bar.find_element(By.XPATH, '//*[@id="module_product_review"]/div/div/div[1]/div[2]/div/div/div[1]/div[1]/span[1]')
    ratings_list.append(rating.text)
    counter = 0
    for i in dilivery_opt:
        if 'Free Delivery' in (i.text):
            counter+=1
            dilivery_status.append(1)
    if(counter==0):
        dilivery_status.append(0)
    
    # Calculate the base score (price)
    base_score = int(price_mobile[index].replace('Rs. ', '').replace(',', ''))

    # Check if the product has "Daraz Mall" status
    daraz_mall_score = 5000 if flagship_mall[index] == 1 else 0

    # Check if the product has free shipping
    free_shipping_score = 1000 if dilivery_status[index] == 1 else 0
    product_rating = float(ratings_list[index])
    seller_rating = float(seller_ratings_list[index].replace('%', ''))
    shipping_performance = float(Shipping_performance[index].replace('%', ''))
    score = base_score + daraz_mall_score + free_shipping_score + 10000 * (
        (0.4 * (product_rating / 5)) + (0.4 * (seller_rating / 100)) + (0.2 * (shipping_performance / 100))
    )
    
    scores.append(score)

    time.sleep(2)  # Add a short delay to allow the page to load


    # Go back to the previous page
    driver.back()
    # Wait for the ad grid to be present again to avoid StaleElementReferenceException
    WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, '//*[@id="root"]/div/div[3]/div/div/div[1]/div[2]')))
# print(price_mobile)
# print(dilivery_status)
# print(ratings_list)
# print(seller_ratings_list)
# print(Shipping_performance)
# print(flagship_mall)
best_product_index = scores.index(max(scores))

# Print the details of the best product
print(f"Best Product Details:")
print(f"Name: {product_name[best_product_index]}")
print(f"Price: {price_mobile[best_product_index]}")
print(f"Daraz Mall Status: {'Yes' if flagship_mall[best_product_index] == 1 else 'No'}")
print(f"Free Shipping: {'Yes' if dilivery_status[best_product_index] == 1 else 'No'}")
print(f"Product Rating: {ratings_list[best_product_index]}")
print(f"Seller Rating: {seller_ratings_list[best_product_index]}")
print(f"Shipping Performance: {Shipping_performance[best_product_index]}")
print(f"Total Score: {scores[best_product_index]}")

Best Product Details:
Name: Xiaomi Redmi Note 12 || 8GB Ram 128GB Rom || 5000mAh Battery 33W Fast Charger
Price: Rs. 51,499
Daraz Mall Status: No
Free Shipping: Yes
Product Rating: 4.7
Seller Rating: 91%
Shipping Performance: 100%
Total Score: 61899.0
